In [1]:
import pandas as pd

In [2]:
CITY = "helsinki"

In [3]:
distance = pd.read_csv(
    f"../output/{CITY}/distance.csv",
    dtype={
        "stop_id": str,
    },
)
ac = pd.read_csv(
    f"../output/{CITY}/amenity_counts_in_accessibility.csv",
    dtype={
        "stop_id": str,
    },
)
pt_ac = pd.read_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv",
    dtype={
        "stop_id": str,
    },
)
stop_geometries = pd.read_csv(
    f"../output/{CITY}/stop_geometries_from_walk.csv",
    dtype={
        "stop_id": str,
    },
)

stop_centralities = pd.read_csv(
    f"../data/stops/{CITY}/stops_with_centrality.csv",
    dtype={
        "stop_id": str,
    },
)
stop_centralities.drop(["Node"], axis=1, inplace=True)
stop_centralities.columns = [
    "eigenvector_centrality",
    "degree_centrality",
    "closeness_centrality",
    "betweenness_centrality",
    "stop_id",
    "cluster",
    "stop_lat",
    "stop_lon",
    "stop_name",
]
stop_centralities.dropna(subset=["stop_id"], inplace=True)

In [4]:
stop_centralities

,eigenvector_centrality,degree_centrality,closeness_centrality,betweenness_centrality,stop_id,cluster,stop_lat,stop_lon,stop_name
0,2.163418e-09,0.000278,0.043629,0.000000,5020503,2,60.976337,25.657594,Lahti
1,2.163418e-09,0.000278,0.043629,0.000000,5020553,2,60.976337,25.657575,Lahti
2,1.127121e-08,0.000557,0.045619,0.000557,5020502,3904,60.647434,25.307804,Mäntsälä
3,1.127121e-08,0.000557,0.045619,0.000557,5020552,3904,60.647434,25.307804,Mäntsälä
4,9.018946e-11,0.000557,0.040139,0.000557,5010505,4,60.631327,24.857754,Hyvinkää
...,...,...,...,...,...,...,...,...,...
7365,4.605070e-22,0.000557,0.020864,0.002780,9212216,3898,60.252130,25.491102,Mjölkviken
7366,2.657502e-14,0.000557,0.028797,0.003329,9300201,3905,60.393460,25.663871,Linja-autoasema
7367,2.657502e-14,0.000557,0.028797,0.003329,9300204,3905,60.393410,25.665047,Linja-autoasema
7368,9.798765e-21,0.000278,0.019263,0.000000,9400206,3906,60.495482,25.535926,Moninkylän ammattik.


In [5]:
wk_amenity = ac.query("costing == 'walk' & range == 15").copy()
wk_amenity.drop(["costing", "range"], axis=1, inplace=True)
wk_amenity.columns = ["stop_id"] + [f"{i}_walk15" for i in wk_amenity.columns[1:]]
mm_amenity = pt_ac.copy()
mm_amenity.drop(["costing", "range"], axis=1, inplace=True)
mm_amenity.columns = ["stop_id"] + [f"{i}_multimodal" for i in mm_amenity.columns[1:]]

In [6]:
m = (
    stop_geometries.drop("geometry", axis=1)
    .merge(distance, on="stop_id")
    .merge(mm_amenity, on="stop_id")
    .merge(wk_amenity, on="stop_id")
    .merge(stop_centralities, on="stop_id")
)
m.to_csv(f"../output/{CITY}/merged.csv", index=False)